<a href="https://colab.research.google.com/github/Alien-Squirrel/fifi/blob/main/gpt2_therapy_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation
Dies könnte je nach System nicht vollständig sein. In der Testumgebung läuft es.

In [14]:
!apt install *tesseract*
!pip uninstall tensorflow && pip install tensorflow-cpu tensorflow
!pip install transformers datasets fsspec==2024.10.0 pytesseract Pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'tesseract-ocr-uzb-cyrl' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-chi-sim' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-chi-tra' for glob '*tesseract*'
Note, selecting 'tesseract-spa-old' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-script-hant-vert' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-script-ethi' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-kat-old' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-script-khmr' for glob '*tesseract*'
Note, selecting 'libtesseract4' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-script-viet' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-kor-vert' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-script-knda' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-jpn-vert' for glob '*tesseract*'
Note, selecting 'tesseract-ocr-aze-cyrl' for glob '*tesseract*'
Note

In [15]:
!pip3 uninstall googletrans
!pip3 install googletrans==3.1.0a0


Found existing installation: googletrans 4.0.2
Uninstalling googletrans-4.0.2:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.11/dist-packages/googletrans-4.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/googletrans/*
Proceed (Y/n)? Y
  Successfully uninstalled googletrans-4.0.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha2

In [2]:
!pip install pymupdf pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.4 MB/s eta 0:00:00


# Datenvorbereitung und Speicherung

In [4]:
import os, pymupdf, time, pytesseract, io, re
from PIL import Image

def is_number(string):
    numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
    for char in string:
        if char not in numbers:
            return False
    return True

with open("input.txt", "w") as f2:
    for (dirpath, dirnames, filenames) in os.walk('./drive/MyDrive/data/pdfs/pdfs/'):
        for f in filenames:
            directory = os.path.join(dirpath, f).split("/")[1:]
            complexity = 0
            chapter = []
            document_name = ""
            document_type = ""
            for i in range (0, len(directory)):
                d = directory[i]
                if i == len(directory) - 1:
                    document_name = ' '.join(f.split(".")[0:-1])
                    document_type = f.split(".")[-1]
                    f2.write("\n")
                elif is_number(d):
                    complexity = d
                else:
                    chapter.append(d)

            print(document_name)
            if document_type == "pdf":
                doc = pymupdf.open(os.path.join(dirpath, f))
                page_count = doc.page_count
                image_texts = []
                for page in doc:
                    for img_index, img in enumerate(page.get_images(full=True)):
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        image = Image.open(io.BytesIO(image_bytes))

                        # OCR auf dem Bild durchführen
                        txt = pytesseract.image_to_string(image)
                        image_texts.append(txt)

                    erlaubte_zeichen = (
                        'abcdefghijklmnopqrstuvwxyz'  # Kleinbuchstaben
                        'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  # Großbuchstaben
                        '0123456789'                  # Zahlen
                        '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '  # Sonderzeichen
                    )

                    # Erstelle eine Regex, die alle Zeichen entfernt, die nicht in der Liste sind
                    regex = f"[^{re.escape(erlaubte_zeichen)}]"
                    complexity = int((page.number / page_count) * 10)
                    text = re.sub(regex, "", page.get_text("text").replace("\n", "") + ". ".join(image_texts))
                    if len(text) > 50:
                        if complexity > 4:
                            f2.write(text + ". This exercise is complex.")
                        elif complexity > 8:
                            f2.write(text  + ". This exercise is very complex")
                        else:
                            f2.write(text)


01 - DBT 6 Levels Of Validation Handout - Interpersonal Effectiveness Skills PDF
02 - DBT ABC PLEASE Handout - Emotion Regulation Skills Pdf
06 - DBT Barriers To Interpersonal Effectiveness Skills Handout
04 - DBT Accumulate Positive Emotions Handout - Emotion Regulation Skills PDF
13 - DBT GIVE Handout - Interpersonal Effectiveness Skills Pdf
07 - DBT Building Mastery Handout - Emotion Regulation Skills PDF
12 - DBT Finding and Making Friends Handout - Social Skills Coping Skills
03 - DBT ACCEPTS Handout - Distress Tolerance Skills PDF-Dialectical Behavior Therapy
05 - DBT Assertiveness In Relationships Handout-Interpersonal Effectiveness Skills
11 - DBT FAST Handout - Interpersonal Effectiveness Skills Pdf
08 - DBT Check The Facts Handout - Emotion Regulation Skills PDF
10 - DBT DEAR MAN Handout - Interpersonal Effectiveness Skills Pdf
14 - DBT Half Smile And Willing Hands Handout - Distress Tolerance Skills PDF
17 - DBT IMPROVE The Moment Handout - Distress Tolerance Skills PDF
20 -

In [8]:
!pip install -U datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#Hier trainiert das neuronale Netz

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# 1. Daten vorbereiten
dataset = load_dataset("text", data_files={"train": "./drive/MyDrive/GPT2 Therapienetzwerk/input.txt"})

# 2. Tokenizer laden
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 hat standardmäßig kein Pad-Token.

# Tokenisierung der Daten
def tokenize_function(examples):
    tokenized_output = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    tokenized_output["labels"] = tokenized_output["input_ids"]  # Setze labels auf input_ids
    return tokenized_output

# Zugriff auf den richtigen Split und Tokenisieren
tokenized_dataset = dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"], num_proc=4)

# 3. Modell laden
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# 4. Trainingsargumente definieren
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/GPT2 Therapienetzwerk/gpt2-finetuned-alpha-1.2",
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=128,
    gradient_accumulation_steps=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True,
    dataloader_num_workers=14,
    load_best_model_at_end=False,  # Bestes Modell nicht laden, da keine Evaluation stattfindet
    eval_strategy="no",  # Keine Evaluation
    save_strategy="steps",  # Speichern alle 500 Schritte
)

# 5. Trainer initialisieren ohne eval_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# 6. Training starten
trainer.train()

# 7. Modell speichern
trainer.save_model("./drive/MyDrive/GPT2 Therapienetzwerk/gpt2-therapy-finetuned-alpha-1.2")
tokenizer.save_pretrained("./drive/MyDrive/GPT2 Therapienetzwerk/gpt2-therapy-finetuned-alpha-1.2")


# Hier ist der Generator
Zeile 8 definiert den Eingabetext. Wenn man eine Frage eingibt, bekommt man Fragen gestellt, aber wenn man stattdessen einen Satzanfang angibt, bekommt man einen Satz oder eine Liste. Man kann nur englisch eingeben, kann allerdings eigenständig ein Übersetzungsprogramm im Internet nutzen.

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1. Tokenizer und Modell laden
tokenizer = GPT2Tokenizer.from_pretrained("./drive/MyDrive/GPT2 Therapienetzwerk/alpha-1.1/gpt2-therapy-finetuned-alpha-1.1")
model = GPT2LMHeadModel.from_pretrained("./drive/MyDrive/GPT2 Therapienetzwerk/alpha-1.1/gpt2-therapy-finetuned-alpha-1.1")

# 2. Eingabetext definieren
input_text = "If someone loses the person should "

# 3. Tokenisierung des Eingabetextes
inputs = tokenizer.encode(input_text, return_tensors="pt")

# 4. Sicherstellen, dass die Eingabe die maximale Länge nicht überschreitet
max_length = 1024
inputs = inputs[:, :max_length]  # Kürzt die Eingabe auf die maximal erlaubte Länge

# 5. Text generieren
output = model.generate(
    inputs,
    max_length=1000,  # Maximale Länge der generierten Sequenz
    num_return_sequences=1,  # Anzahl der generierten Sequenzen
    no_repeat_ngram_size=2,  # Verhindert Wiederholungen von n-Grammen
    temperature=0.7,  # Steuert die Zufälligkeit der Ausgabe
    top_k=50,  # Wählt die Top-k-Wahrscheinlichkeiten für die nächsten Tokens
    top_p=0.95,  # Nucleus Sampling
    do_sample=True,  # Aktiviert Sampling
    early_stopping=False,  # Stoppt die Generierung, wenn die End-of-Sequence-Token erreicht ist
)

# 6. Ausgabe dekodieren
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 7. Ausgabe anzeigen
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


If someone loses the person should  hold them in place  until they are able to walk away.If you lose your child  you need to look for a caregiver who can help you get through this difficult time.
